**This notebook based on an exercise in the [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/xgboost).**

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv")

In [3]:
# Read the data
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [4]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = XGBRegressor(n_estimators=1000, learning_rate=0.05)

In [5]:
# Make a copy to avoid changing original data
X_valid_eval=X_valid.copy()
# Remove the model from pipeline
eval_set_pipe = Pipeline(steps = [('preprocessor', preprocessor)])
# fit transform X_valid.copy()
X_valid_eval = eval_set_pipe.fit(X_train, y_train).transform (X_valid_eval)

In [6]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train,
                model__early_stopping_rounds=50,
                model__eval_set=[(X_valid_eval, y_valid)]
)


# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

[0]	validation_0-rmse:190309.26562
[1]	validation_0-rmse:181580.92188
[2]	validation_0-rmse:173377.75000
[3]	validation_0-rmse:165566.65625
[4]	validation_0-rmse:158257.42188
[5]	validation_0-rmse:151197.42188
[6]	validation_0-rmse:144560.06250
[7]	validation_0-rmse:138040.31250
[8]	validation_0-rmse:131965.82812
[9]	validation_0-rmse:126296.07031
[10]	validation_0-rmse:120776.16406
[11]	validation_0-rmse:115432.46094
[12]	validation_0-rmse:110737.27344
[13]	validation_0-rmse:106009.87500
[14]	validation_0-rmse:101745.64062
[15]	validation_0-rmse:97683.46875
[16]	validation_0-rmse:93946.13281
[17]	validation_0-rmse:90396.86719
[18]	validation_0-rmse:86762.33594
[19]	validation_0-rmse:83471.14062
[20]	validation_0-rmse:80557.77344
[21]	validation_0-rmse:77723.14844
[22]	validation_0-rmse:75003.50000
[23]	validation_0-rmse:72533.73438
[24]	validation_0-rmse:70256.07031
[25]	validation_0-rmse:68084.07031
[26]	validation_0-rmse:65798.43750
[27]	validation_0-rmse:63855.09766
[28]	validation

In [7]:
preds_test = my_pipeline.predict(X_test)
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)